# Feature Pruning and Hyper Parameter Tuning

**`HYPOTHESIS`** By removing "dead-weight" features, the model will be faster and simpler. The removal of noise may lead to a slight increase in CV score and a potential increase in stability - measured by std. deviation

## Setup

In [1]:
#### TEMPORARY
import sys
sys.path.append('/home/bac/code/kaggle/kaggle-cmi-detect-behavior/')

In [2]:
import pandas as pd
import numpy as np
import os
import catboost as cat
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

# --- Pathing and Experiment Tracking Setup ---
from src.tracking import ExperimentTracker
from src.config import DATA_PATH, PROJECT_PATH, USE_WANDB, WANDB_PROJECT, WANDB_ENTITY

tracker = ExperimentTracker(
    project_path=os.path.expanduser(PROJECT_PATH),
    use_wandb=USE_WANDB,
    wandb_project_name=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY
)

SEED = 42

In [3]:
def display_all(df):
    with pd.option_context("display.max_rows", 500, "display.max_columns", 1000):
        display(df)

In [4]:
# Load the raw data and create helper maps
train_sensor = pd.read_csv(os.path.join(os.path.expanduser(DATA_PATH), 'train.csv'))
train_demos = pd.read_csv(os.path.join(os.path.expanduser(DATA_PATH), 'train_demographics.csv'))
train_df = pd.merge(train_sensor, train_demos, on='subject', how='left')
metadata = train_df[['gesture', 'sequence_type']].drop_duplicates()
gesture_to_seq_type_map = metadata.set_index('gesture')['sequence_type'].to_dict()
gesture_map = {label: i for i, label in enumerate(metadata['gesture'].unique())}
inv_gesture_map = {i: label for label, i in gesture_map.items()}

In [5]:
# Load features feather file and permutation importance csv
features_w4_df = pd.read_feather("outputs/features_4n.fea")
perm_importance_df = pd.read_csv("outputs/permutation_importance_W4n.csv")

features_w4_df.head()

,sequence_id,subject,gesture,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm,acc_x_mean_Gesture,acc_x_mean_Transition,acc_x_std_Gesture,acc_x_std_Transition,acc_x_min_Gesture,acc_x_min_Transition,acc_x_max_Gesture,acc_x_max_Transition,acc_x_skew_Gesture,acc_x_skew_Transition,acc_y_mean_Gesture,acc_y_mean_Transition,acc_y_std_Gesture,acc_y_std_Transition,acc_y_min_Gesture,acc_y_min_Transition,acc_y_max_Gesture,acc_y_max_Transition,acc_y_skew_Gesture,acc_y_skew_Transition,acc_z_mean_Gesture,acc_z_mean_Transition,acc_z_std_Gesture,acc_z_std_Transition,acc_z_min_Gesture,acc_z_min_Transition,acc_z_max_Gesture,acc_z_max_Transition,acc_z_skew_Gesture,acc_z_skew_Transition,rot_w_mean_Gesture,rot_w_mean_Transition,rot_w_std_Gesture,rot_w_std_Transition,rot_w_min_Gesture,rot_w_min_Transition,rot_w_max_Gesture,rot_w_max_Transition,rot_w_skew_Gesture,rot_w_skew_Transition,rot_x_mean_Gesture,rot_x_mean_Transition,rot_x_std_Gesture,rot_x_std_Transition,rot_x_min_Gesture,rot_x_min_Transition,rot_x_max_Gesture,rot_x_max_Transition,rot_x_skew_Gesture,rot_x_skew_Transition,rot_y_mean_Gesture,rot_y_mean_Transition,rot_y_std_Gesture,rot_y_std_Transition,rot_y_min_Gesture,rot_y_min_Transition,rot_y_max_Gesture,rot_y_max_Transition,rot_y_skew_Gesture,rot_y_skew_Transition,rot_z_mean_Gesture,rot_z_mean_Transition,rot_z_std_Gesture,rot_z_std_Transition,rot_z_min_Gesture,rot_z_min_Transition,rot_z_max_Gesture,rot_z_max_Transition,rot_z_skew_Gesture,rot_z_skew_Transition,thm_1_mean_Gesture,thm_1_mean_Transition,thm_1_std_Gesture,thm_1_std_Transition,thm_1_min_Gesture,thm_1_min_Transition,thm_1_max_Gesture,thm_1_max_Transition,thm_1_skew_Gesture,thm_1_skew_Transition,thm_2_mean_Gesture,thm_2_mean_Transition,thm_2_std_Gesture,thm_2_std_Transition,thm_2_min_Gesture,thm_2_min_Transition,thm_2_max_Gesture,thm_2_max_Transition,thm_2_skew_Gesture,thm_2_skew_Transition,thm_3_mean_Gesture,thm_3_mean_Transition,thm_3_std_Gesture,thm_3_std_Transition,thm_3_min_Gesture,thm_3_min_Transition,thm_3_max_Gesture,thm_3_max_Transition,thm_3_skew_Gesture,thm_3_skew_Transition,thm_4_mean_Gesture,thm_4_mean_Transition,thm_4_std_Gesture,thm_4_std_Transition,thm_4_min_Gesture,thm_4_min_Transition,thm_4_max_Gesture,thm_4_max_Transition,thm_4_skew_Gesture,thm_4_skew_Transition,thm_5_mean_Gesture,thm_5_mean_Transition,thm_5_std_Gesture,thm_5_std_Transition,thm_5_min_Gesture,thm_5_min_Transition,thm_5_max_Gesture,thm_5_max_Transition,thm_5_skew_Gesture,thm_5_skew_Transition,acc_mag_mean_Gesture,acc_mag_mean_Transition,acc_mag_std_Gesture,acc_mag_std_Transition,acc_mag_min_Gesture,acc_mag_min_Transition,acc_mag_max_Gesture,acc_mag_max_Transition,acc_mag_skew_Gesture,acc_mag_skew_Transition,rot_mag_mean_Gesture,rot_mag_mean_Transition,rot_mag_std_Gesture,rot_mag_std_Transition,rot_mag_min_Gesture,rot_mag_min_Transition,rot_mag_max_Gesture,rot_mag_max_Transition,rot_mag_skew_Gesture,rot_mag_skew_Transition,jerk_mean_Gesture,jerk_mean_Transition,jerk_std_Gesture,jerk_std_Transition,jerk_min_Gesture,jerk_min_Transition,jerk_max_Gesture,jerk_max_Transition,jerk_skew_Gesture,jerk_skew_Transition,thm_grad_1_2_mean_Gesture,thm_grad_1_2_mean_Transition,thm_grad_1_2_std_Gesture,thm_grad_1_2_std_Transition,thm_grad_1_2_min_Gesture,thm_grad_1_2_min_Transition,thm_grad_1_2_max_Gesture,thm_grad_1_2_max_Transition,thm_grad_1_2_skew_Gesture,thm_grad_1_2_skew_Transition,thm_grad_2_3_mean_Gesture,thm_grad_2_3_mean_Transition,thm_grad_2_3_std_Gesture,thm_grad_2_3_std_Transition,thm_grad_2_3_min_Gesture,thm_grad_2_3_min_Transition,thm_grad_2_3_max_Gesture,thm_grad_2_3_max_Transition,thm_grad_2_3_skew_Gesture,thm_grad_2_3_skew_Transition,thm_grad_3_4_mean_Gesture,thm_grad_3_4_mean_Transition,thm_grad_3_4_std_Gesture,thm_grad_3_4_std_Transition,thm_grad_3_4_min_Gesture,thm_grad_3_4_min_Transition,thm_grad_3_4_max_Gesture,thm_grad_3_4_max_Transition,thm_grad_3_4_skew_Gesture,thm_grad_3_4_skew_Transition,thm_grad_4_5_mean_Gesture,thm_grad_4_5_mean_Transition,thm_grad_4_5_s

In [6]:
perm_importance_df.head()

,feature,importance_mean
0,acc_z_std_Gesture,0.011634
1,acc_y_std_Gesture,0.008877
2,acc_x_std_Gesture,0.004547
3,thm_2_max_Gesture,0.003645
4,thm_2_mean_Gesture,0.003379


In [7]:
features_w4_df.shape, perm_importance_df.shape

((8151, 347), (343, 2))

## Pipeline

In [8]:
def average_f1_score(y_true_encoded, y_pred_proba):
    """
    Calculates the official competition F1 score.
    
    Args:
        y_true_encoded: True labels, integer encoded.
        y_pred_proba: Predicted probabilities from the model.
    """
    # Get predicted labels by finding the class with the highest probability
    y_pred_encoded = np.argmax(y_pred_proba, axis=1)
    
    # Map integer-encoded labels back to the string representations
    y_true_str = pd.Series(y_true_encoded).map(inv_gesture_map)
    y_pred_str = pd.Series(y_pred_encoded).map(inv_gesture_map)
    
    # Binary F1
    y_true_binary = y_true_str.map(gesture_to_seq_type_map)
    y_pred_binary = y_true_str.map(gesture_to_seq_type_map)
    binary_f1 = f1_score(y_true_binary, y_pred_binary, pos_label='Target', average='binary')
    
    # Macro F1 (collaped non-target class)
    def collapse_non_target(gesture):
        return 'non_target' if gesture_to_seq_type_map[gesture] == 'Non-Target' else gesture
    
    y_true_collapsed = y_true_str.apply(collapse_non_target)
    y_pred_collapsed = y_pred_str.apply(collapse_non_target)
    macro_f1 = f1_score(y_true_collapsed, y_pred_collapsed, average='macro')
    
    # Final score = average of the two components
    return (binary_f1 + macro_f1) / 2

## Pruning + Training

In [10]:
# Identify features with positive importance
keep_features = perm_importance_df[perm_importance_df['importance_mean'] > 0]['feature'].tolist()

# Target and grouping columns need to be added to the features list
keep_cols = ['sequence_id', 'gesture', 'gesture_encoded'] + keep_features

# Create new, pruned dataframe
pruned_features = features_w4_df[keep_cols]

print(f"Original number of features: {len(features_w4_df.columns) - 3}")
print(f"Number of features after pruning: {len(pruned_features.columns) - 3}")
print("Pruned features DataFrame created successfully.")

Original number of features: 344
Number of features after pruning: 76
Pruned features DataFrame created successfully.


In [11]:
# Configuration 
FEATURE_WAVE = "Wave 4 (Pruned)"
MODEL_NAME = "CatBoost"
EXPERIMENT_NAME = f"{FEATURE_WAVE}-{MODEL_NAME}-CPU"
N_SPLITS = 5
SEED = 42

# Model Parameters
params = {
    'iterations': 1000, 'learning_rate': 0.05, 'depth': 6,
    'loss_function': 'MultiClass', 'eval_metric': 'MultiClass',
    'random_seed': SEED, 'verbose': 0
}

## Prepare data for CV ---
X = pruned_features.drop(columns=['sequence_id', 'gesture', 'gesture_encoded'])
y = pruned_features['gesture_encoded']

# Get subject from the un-pruned dataframe for CV
groups = features_w4_df['subject']

In [12]:
# Cross-Validation Loop
print("\n" + "="*50)
print(f"Running CV for {MODEL_NAME} on {FEATURE_WAVE} Features")
print("="*50)

fold_scores = []
cv = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    model = cat.CatBoostClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100)
    
    val_preds_proba = model.predict_proba(X_val)
    fold_score = average_f1_score(y_val, val_preds_proba)
    fold_scores.append(fold_score)
    print(f"Fold {fold+1} Competition F1 Score: {fold_score:.5f}")

# --- Final Score and Logging ---
mean_cv_score = np.mean(fold_scores)
print(f"\n--- CV Summary for {MODEL_NAME} on {FEATURE_WAVE} Features ---")
print(f"Mean Competition F1 Score: {mean_cv_score:.5f}")
print(f"Std Dev: {np.std(fold_scores):.5f}\n")

tracker.log_experiment(
    experiment_name=EXPERIMENT_NAME, model_name=MODEL_NAME, feature_wave=FEATURE_WAVE,
    cv_score=mean_cv_score, params=params,
    notes="Re-trained Waved 4 (fixed) CatBoost on a pruned feature set (Permutation Importance > 0)."
)


Running CV for CatBoost on Wave 4 (Pruned) Features
--- Fold 1/5 ---
Fold 1 Competition F1 Score: 0.82885
--- Fold 2/5 ---
Fold 2 Competition F1 Score: 0.77407
--- Fold 3/5 ---
Fold 3 Competition F1 Score: 0.79894
--- Fold 4/5 ---
Fold 4 Competition F1 Score: 0.77499
--- Fold 5/5 ---
Fold 5 Competition F1 Score: 0.78693

--- CV Summary for CatBoost on Wave 4 (Pruned) Features ---
Mean Competition F1 Score: 0.79275
Std Dev: 0.02020

Experiment 'Wave 4 (Pruned)-CatBoost-CPU' logged to /home/bac/code/kaggle/kaggle-cmi-detect-behavior/experiment_log.csv


wandb: Currently logged in as: b-a-chaudhry (b-a-chaudhry-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


cv_score,▁
cv_score,0.79275
feature_wave,Wave 4 (Pruned)
model_name,CatBoost


Experiment 'Wave 4 (Pruned)-CatBoost-CPU' logged to W&B


**`ANALYSIS OF PRUNING`**

- The Full Set is Superior: Despite a 77% reduction in feature count, the pruned model's performance decreased by a significant -0.01097.

- The "Weak Features as Regularizers" Hypothesis:  The 268 features with low or zero permutation importance were not useless. In aggregate, they acted as a form of regularization, helping the model generalize better and preventing it from overfitting to the more dominant signals. Removing them made the model simpler but ultimately less accurate on this specific task.

- Final Verdict on Feature Engineering:  The evidence is conclusive: the full Wave 4 - Fixed feature set is our champion. We can now proceed with very high confidence that we have the best possible set of "ingredients" for our model.

## Hyper-Parameter Tuning

We will be using the [`Optuna`](https://optuna.org/#key_features) library to automate the search for the best model configuration.

In [17]:
# Define the Optuna objective function - single trial

def objective(trial, X, y, groups, scorer):
    """
    Defines a single trial for Optuna. It suggests hyperparameters,
    trains a CatBoost model using them, and returns the mean CV F1 score.
    """
    params = {
        'iterations': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.08, log=True),
        'depth': trial.suggest_int('depth', 5, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2.0, 15.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 1.0, 10.0, log=True),
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'random_seed': SEED,
        'verbose': 0
    }

    cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=SEED)
    fold_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
        
        model = cat.CatBoostClassifier(**params)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=150, verbose=0)
        
        val_preds_proba = model.predict_proba(X_val)
        
        fold_score = scorer(y_val, val_preds_proba)
        fold_scores.append(fold_score)
        
        trial.report(fold_score, fold)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return np.mean(fold_scores)

In [18]:
# --- Prepare data for tuning ---
X = features_w4_df.drop(columns=['sequence_id', 'subject', 'gesture', 'gesture_encoded'])
y = features_w4_df['gesture_encoded']
groups = features_w4_df['subject']

In [ ]:
# Create the study object with direction set to 'maximize' for our F1 score
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner() # Review other options - FLAG
)

scorer = lambda y_true, y_pred: average_f1_score(y_true, y_pred)
objective_with_scorer = lambda trial: objective(trial, X, y, groups, scorer)

# Run the optimization with the corrected objective.
study.optimize(objective_with_scorer, n_trials=50, show_progress_bar=True)

# --- Print and Log the Best Results ---
print("\n" + "="*50)
print("Optuna Hyperparameter Tuning Complete")
print(f"Number of finished trials: {len(study.trials)}")

# Select the best trial
best_trial = study.best_trial
print(f"Best trial's F1 Score: {best_trial.value:.5f}")
print("Best trial's hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# Log the best trial as our new champion experiment
tracker.log_experiment(
    experiment_name="CatBoost-Tuned-Wave4-Features",
    model_name="CatBoost (Tuned)",
    feature_wave="Wave 4 (Fixed)",
    cv_score=best_trial.value,
    params=best_trial.params,
    notes="Best parameters found after 50 Optuna trials on the Wave 4 feature set."
)

**`POORLY OPTIMIZED TUNING SETUP`**

- In its current form, the Optuna setup would take approximately 84+ hours to run trials on various hyper-parameter tests. This is totally unacceptable.
- This part of the modeling process has been moved to notebook `05A-hyper-parameter.ipynb` on Colab.
- Instead of re-splitting the data 5 times for every trial, we will perform one single split at the very beginning using a large training set (~80% of the data) and a single, fixed hold-out validation set (~20%).
- Inside each Optuna trial, the model will train once on the large training set and evaluate once on the hold-out set. This immediately gives us a 5x speedup.
- Running these optimizations on a CPU is extremely time consuming, so we will leverage Catboost's internal GPU support while leveraging GPU compute in Colab to achieve another 2-5x in speedups.

Based on Colab's hyper-parameter tuning, these are the results based on the fixed hold out approach:

```

Optuna Hyperparameter Tuning Complete
Number of finished trials: 75
Best trial's Hold-Out F1 Score: 0.84691
Best trial's hyperparameters:
    learning_rate: 0.0676968816733559
    depth: 5
    l2_leaf_reg: 2.115556787694446
    bagging_temperature: 0.195243584852945
    random_strength: 1.0061621160663667

```

## Hyper-Parameter Optimized (Fixed-Hold Out) Run 

In [10]:
FEATURE_WAVE = "Wave 4 (Fixed)"
MODEL_NAME = "CatBoost (Final-Tuned)"
EXPERIMENT_NAME = f"Final-Validation-{MODEL_NAME}-CPU"
N_SPLITS = 5
SEED = 42

# Using best performing parameters from the Optuna trials

best_params = {
    'iterations': 2000, # Increased iterations to allow early stopping to work
    'learning_rate': 0.0676968816733559,
    'depth': 5,
    'l2_leaf_reg': 2.115556787694446,
    'bagging_temperature': 0.195243584852945,
    'random_strength': 1.0061621160663667,
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'random_seed': SEED,
    'task_type' : 'GPU',
    'verbose': 0
}

# Prepare data for CV 
X = features_w4_df.drop(columns=['sequence_id', 'subject', 'gesture', 'gesture_encoded'])
y = features_w4_df['gesture_encoded']
groups = features_w4_df['subject']

# Cross-Validation Loop 
print("\n" + "="*50)
print(f"Running Final Validation for {MODEL_NAME} on {FEATURE_WAVE} Features")
print("="*50)

fold_scores = []
cv = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups)):
    print(f"--- Fold {fold+1}/{N_SPLITS} ---")
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    # Instantiate the model with our best parameters
    model = cat.CatBoostClassifier(**best_params)
    
    # Train with early stopping
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=150)
    
    # Predict and score
    val_preds_proba = model.predict_proba(X_val)
    fold_score = average_f1_score(y_val, val_preds_proba)
    fold_scores.append(fold_score)
    print(f"Fold {fold+1} Competition F1 Score: {fold_score:.5f}")

# --- Final Score and Logging ---
mean_cv_score = np.mean(fold_scores)
print(f"\n--- Final CV Summary for Tuned {MODEL_NAME} ---")
print(f"Mean Competition F1 Score: {mean_cv_score:.5f}")
print(f"Std Dev: {np.std(fold_scores):.5f}\n")

tracker.log_experiment(
    experiment_name=EXPERIMENT_NAME, 
    model_name=MODEL_NAME, 
    feature_wave=FEATURE_WAVE,
    cv_score=mean_cv_score, 
    params=best_params,
    notes="Validation of best Optuna parameters using 5-fold StratifiedGroupKFold CV."
)



Running Final Validation for CatBoost (Final-Tuned) on Wave 4 (Fixed) Features
--- Fold 1/5 ---
Fold 1 Competition F1 Score: 0.84691
--- Fold 2/5 ---
Fold 2 Competition F1 Score: 0.77755
--- Fold 3/5 ---
Fold 3 Competition F1 Score: 0.80802
--- Fold 4/5 ---
Fold 4 Competition F1 Score: 0.79580
--- Fold 5/5 ---
Fold 5 Competition F1 Score: 0.80530

--- Final CV Summary for Tuned CatBoost (Final-Tuned) ---
Mean Competition F1 Score: 0.80671
Std Dev: 0.02276

Experiment 'Final-Validation-CatBoost (Final-Tuned)-CPU' logged to /home/bac/code/kaggle/kaggle-cmi-detect-behavior/experiment_log.csv


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


wandb: Network error (ConnectionError), entering retry loop.


cv_score,▁
cv_score,0.80671
feature_wave,Wave 4 (Fixed)
model_name,CatBoost (Final-Tune...


Experiment 'Final-Validation-CatBoost (Final-Tuned)-CPU' logged to W&B


## Train Final Model on 100% Training Data and Validate With the Test Set

The results from an intensive 50 trial Stratified K-Fold [Optuna tuning run on Colab](https://github.com/bachaudhry/kaggle-cmi-detect-behavior/blob/main/notebooks/05B-hyper-parameter-tuning.ipynb) indicate that the combination of hyper-parameters below result in an improvement of **+.00783** mean CV F1 score compared to the Wave 4 baseline:


    Final Robust Optuna Tuning Complete
    Number of finished trials: 50
    Best trial's Mean CV F1 Score: 0.81155
    Best trial's hyperparameters:
        
        learning_rate: 0.06442938671410496
        depth: 6
        l2_leaf_reg: 4.3544165839313385
        bagging_temperature: 0.13574803789441542
        random_strength: 7.435358758357957

We will now train the final model (of this leg of the development process) on the entirety of the training data while prepping to make our submission to Kaggle.

In [9]:
FEATURE_WAVE = "Wave 4 (Fixed)"
MODEL_NAME = "CatBoost Wave-4 Tuned (Final)"
EXPERIMENT_NAME = f"Final-Model-Training"

# final features are already loaded

# Best Hyperparameters from Final Optuna Study 
best_params = {
    'iterations': 2000, 
    'learning_rate': 0.06442938671410496,
    'depth': 6,
    'l2_leaf_reg': 4.3544165839313385,
    'bagging_temperature': 0.13574803789441542,
    'random_strength': 7.435358758357957,
    'loss_function': 'MultiClass',
    'eval_metric': 'MultiClass',
    'random_seed': SEED,
    'task_type' : 'GPU', # Speed up training using GPU
    'verbose': 100
}

In [20]:
X_train_full = features_w4_df.drop(columns=['sequence_id', 'subject', 'gesture', 'gesture_encoded'])
y_train_full = features_w4_df['gesture_encoded']

print("\n" + "="*50)
print(f"Training Final Model on ALL Training Data")
print(f"Features: {FEATURE_WAVE}")
print("="*50)

final_model = cat.CatBoostClassifier(**best_params)

final_model.fit(X_train_full, y_train_full, plot=True)

# Save the model artifact
MODELS_PATH = os.path.join(os.path.expanduser(PROJECT_PATH), 'models')
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)
    
model_save_path = os.path.join(MODELS_PATH, 'catboost_final_model.cbm')
final_model.save_model(model_save_path)
    
print(f"Model saved to: {model_save_path}")

# Log training event

tracker.log_experiment(
    experiment_name=EXPERIMENT_NAME, 
    model_name=MODEL_NAME, 
    feature_wave=FEATURE_WAVE,
    cv_score=0.81155, # Log our best validated CV score for this model
    params=best_params,
    notes="Final model trained on complete training data. Saved to file for submission."
)


Training Final Model on ALL Training Data
Features: Wave 4 (Fixed)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 2.7471977	total: 55.2ms	remaining: 1m 50s
100:	learn: 1.2250646	total: 1.34s	remaining: 25.2s
200:	learn: 0.9171906	total: 2.78s	remaining: 24.9s
300:	learn: 0.7106409	total: 4.61s	remaining: 26s
400:	learn: 0.5723588	total: 6.44s	remaining: 25.7s
500:	learn: 0.4767190	total: 8.27s	remaining: 24.7s
600:	learn: 0.4044465	total: 10.1s	remaining: 23.5s
700:	learn: 0.3465841	total: 11.9s	remaining: 22.1s
800:	learn: 0.2989821	total: 13.8s	remaining: 20.6s
900:	learn: 0.2602891	total: 15.6s	remaining: 19s
1000:	learn: 0.2288687	total: 17.4s	remaining: 17.3s
1100:	learn: 0.2023758	total: 19.2s	remaining: 15.6s
1200:	learn: 0.1810780	total: 20.9s	remaining: 13.9s
1300:	learn: 0.1615314	total: 22.8s	remaining: 12.2s
1400:	learn: 0.1447627	total: 24.6s	remaining: 10.5s
1500:	learn: 0.1304741	total: 26.3s	remaining: 8.76s
1600:	learn: 0.1182468	total: 28.1s	remaining: 7.01s
1700:	learn: 0.1073759	total: 29.9s	remaining: 5.26s
1800:	learn: 0.0981724	total: 31.8s	remaining: 3.51s
1900:	l

wandb: Currently logged in as: b-a-chaudhry (b-a-chaudhry-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


cv_score,▁
cv_score,0.81155
feature_wave,Wave 4 (Fixed)
model_name,CatBoost Wave-4 Tune...


Experiment 'Final-Model-Training' logged to W&B
